# Linear Regression

In [6]:
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 1000

%matplotlib inline

In [2]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

## Preprocessing

### Function to convert minutes to seconds

In [3]:
def calc_secs(x):
    return int(x.split(':')[0])*60 + int(x.split(':')[1])

### Function to calculate rolling statistics for players from box score data

In [9]:
def formatted_player_stats(table, cols, year = '', lag = None):
    ## Pass year as a string to calculate stats for a given season
    if year:
        year_clean = '00' + year[0] + year[2:]
        select_statement = "select * from nba.{} where (GAME_ID REGEXP '(^{})') order by GAME_ID".format(table, year_clean)
    else:
        select_statement = "SELECT * FROM nba.{} order by GAME_ID;".format(table)
    
    cursor.execute(select_statement)
    nba_data = cursor.fetchall()
    
    field_names = [i[0] for i in cursor.description]
    nba_df.columns = field_names
    
    ### Drop games where players do not play
    nba_df_played = nba_df[nba_df['MIN'].notnull()]
    
    ### Calculate seconds played in a game
    nba_df_played['SEC'] = nba_df_played['MIN'].apply(lambda x : calc_secs(x))
    
    ### For each player, calculate expanding mean and rolling mean for each specified statistic in cols arguement 
    players = []
    cols = cols
    for player in nba_df_played['PLAYER_ID'].unique():
        df = nba_df_played[nba_df_played['PLAYER_ID']==player]
        for col in cols:
            df['expanding_{}'.format(col)] = df.expanding().mean()[col].shift(1)
            if lag:
                df['rolling_{}'.format(col+str(lag))] = df.rolling(lag).mean()[col].shift(1)
        df.dropna(inplace=True)
        players.append(df)
    return pd.concat(players)